In [23]:
import math
import csv
import pvlib
import os
import sys

sys.path.insert(0, '../bifacialvf')

#from vf import getBackSurfaceIrradiances, getFrontSurfaceIrradiances, getGroundShadeFactors
#from vf import getSkyConfigurationFactors, trackingBFvaluescalculator, rowSpacing
#from sun import hrSolarPos, perezComp, solarPos, sunIncident
import bifacialvf   
from loadVFresults import loadVFresults
#from bifacialvf import simulate
import pandas as pd

print "Loaded"

Loaded


In [30]:
# SETUP 1
compiledwritefiletitle="..\\bifacialvf\data\Output\RichmondVA_COMPILED_VF.csv"
TMYtoread = "..\\bifacialvf\data\\724010TYA.csv"   # VA Richmond
lat = 37.517                  # Latitude(deg)
lng = -77.317                # Longitude(deg)
tz = -5                     # timezone
beta = 10                   # PV tilt (deg)
sazm = 180                  # PV Azimuth(deg)
C = 0.15                      # GroundClearance(panel slope lengths)
rowType = "interior"        # RowType(first interior last single)
transFactor = 0.013         # TransmissionFactor(open area fraction)
PVfrontSurface = "glass"    #PVfrontSurface(glass or AR glass)
PVbackSurface = "glass"     # PVbackSurface(glass or AR glass)
offset = 0                  # RefCellOffsetFromBack(panel slope lengths)
albedo = 0.62               # albedo
dataInterval = 60           # DataInterval(minutes)
portraitorlandscape='landscape'  # Panel Orientation (portrait landscape ) 
cellRows = 6                # CellRows(# hor rows in panel)   <-->
#D  0.51519                 # DistanceBetweenRows(panel slope lengths)
rtr = 1.5
tracking=False; backtrack=False
D = None


albedoloop=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 0.9]
Cloop=[0.15, 0.2, 0.25, 0.4, 0.5, 0.6, 0.75, 0.82, 1,2]
Rtrloop=[1, 1.5, 1.75, 2.25, 2.5, 3]
betaloop=[5, 10, 15, 20, 25, 30, 35, 40]

with open (compiledwritefiletitle,'wb') as csvfile:
    sw = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)

    outputheader=['Run', 'Value', 'Front Irr', 'Backk Irr', 'BackIrradRatio']
            
    sw.writerow(outputheader)
    
    for ii in range (0, 4):

        if ii==0:
            ctloop=Cloop
            ctlooptitle='height'            
        if ii==1:
            ctloop=Rtrloop
            ctlooptitle='rtr'           
        if ii==2:
            ctloop=betaloop
            ctlooptitle='beta'
        if ii==3:
            ctloop=albedoloop
            ctlooptitle='albedo'          


        for jj in range (0, len(ctloop)):

            outputvalues=[]
            # Base params
            albedo=0.62
            C=0.15
            beta=10
            rtr=1.5
           
            if ii==0:
                C=ctloop[jj]                
            if ii==1:
                rtr=ctloop[jj]
            if ii==2:
                beta=ctloop[jj]                
            if ii==3:
                albedo=ctloop[jj]    


        #   writefiletitle="data/Output/SWLandscape6Sensors/"+"OTFROOF_Landscape609Sensors_"+city+"_"+str(ctlooptitle)+"_"+str(ctloop[jj])+".csv"
        #    writefiletitle="data/Output/RICHMOND_"+"_"+str(ctlooptitle)+"_"+str(ctloop[jj])+".csv"

            bifacialvf.simulate(TMYtoread, writefiletitle, beta, sazm, 
                            C, D, rowType=rowType, transFactor=transFactor, cellRows=cellRows,
                            PVfrontSurface=PVfrontSurface, PVbackSurface=PVbackSurface, albedo=albedo, 
                            tracking=tracking, backtrack=backtrack, rtr=rtr)

            #4. Load the results from the resultfile
            (data, metadata) = loadVFresults(writefiletitle)
            #print data.keys()
            # calculate average front and back global tilted irradiance across the module chord
            data['GTIFrontavg'] = data[['No_1_RowFrontGTI', 'No_2_RowFrontGTI','No_3_RowFrontGTI','No_4_RowFrontGTI','No_5_RowFrontGTI','No_6_RowFrontGTI']].mean(axis=1)
            data['GTIBackavg'] = data[['No_1_RowBackGTI', 'No_2_RowBackGTI','No_3_RowBackGTI','No_4_RowBackGTI','No_5_RowBackGTI','No_6_RowBackGTI']].mean(axis=1)

            # Print the annual bifacial ratio
            frontIrrSum = data['GTIFrontavg'].sum()
            backIrrSum = data['GTIBackavg'].sum()
            print('The bifacial ratio for ground clearance {} and rtr spacing {} is: {:.1f}%'.format(C,rtr,backIrrSum/frontIrrSum*100))

            outputvalues.append(ctlooptitle)
            outputvalues.append(ctloop[jj])
            outputvalues.append(frontIrrSum)
            outputvalues.append(backIrrSum)
            outputvalues.append(backIrrSum/frontIrrSum*100)
        
            sw.writerow(outputvalues)

            
print ""
print "FINISHED FINISHED"

 
********* 
Running Simulation for TMY3:  ..\bifacialvf\data\724010TYA.csv
Location:   "RICHMOND INTERNATIONAL AP"
Lat:  37.517  Long:  -77.317  Tz  -5.0
Parameters: beta:  10   Sazm:  180   Height:  0.15   rtr separation:  1.5   Row type:  interior   Albedo:  0.62
Saving into data/Output/RICHMOND__height_0.15.csv
 
 
Distance between rows for no shading on Dec 21 at 9 am solar time =  0.451287350654
Actual distance between rows =  0.515192246988
 
Finished
The bifacial ratio for ground clearance 0.15 and rtr spacing 1.5 is: 10.8%
 
********* 
Running Simulation for TMY3:  ..\bifacialvf\data\724010TYA.csv
Location:   "RICHMOND INTERNATIONAL AP"
Lat:  37.517  Long:  -77.317  Tz  -5.0
Parameters: beta:  10   Sazm:  180   Height:  0.2   rtr separation:  1.5   Row type:  interior   Albedo:  0.62
Saving into data/Output/RICHMOND__height_0.2.csv
 
 
Distance between rows for no shading on Dec 21 at 9 am solar time =  0.451287350654
Actual distance between rows =  0.515192246988
 
Finished
T

Finished
The bifacial ratio for ground clearance 0.15 and rtr spacing 3 is: 13.4%
 
********* 
Running Simulation for TMY3:  ..\bifacialvf\data\724010TYA.csv
Location:   "RICHMOND INTERNATIONAL AP"
Lat:  37.517  Long:  -77.317  Tz  -5.0
Parameters: beta:  5   Sazm:  180   Height:  0.15   rtr separation:  1.5   Row type:  interior   Albedo:  0.62
Saving into data/Output/RICHMOND__beta_5.csv
 
 
Distance between rows for no shading on Dec 21 at 9 am solar time =  0.226505597509
Actual distance between rows =  0.503805301908
 
Finished
The bifacial ratio for ground clearance 0.15 and rtr spacing 1.5 is: 10.8%
 
********* 
Running Simulation for TMY3:  ..\bifacialvf\data\724010TYA.csv
Location:   "RICHMOND INTERNATIONAL AP"
Lat:  37.517  Long:  -77.317  Tz  -5.0
Parameters: beta:  10   Sazm:  180   Height:  0.15   rtr separation:  1.5   Row type:  interior   Albedo:  0.62
Saving into data/Output/RICHMOND__beta_10.csv
 
 
Distance between rows for no shading on Dec 21 at 9 am solar time =  

Finished
The bifacial ratio for ground clearance 0.15 and rtr spacing 1.5 is: 15.6%

FINISHED FINISHED
